In [ ]:
import pandas as pd
import gensim
import numpy as np
import torch

### 70. 単語ベクトルの和による特徴量

In [ ]:
train = pd.read_csv('data/chap6/train.txt',sep='\t',header=None)
valid = pd.read_csv('data/chap6/valid.txt',sep='\t',header=None)
test = pd.read_csv('data/chap6/test.txt',sep='\t',header=None)

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('data/chap7/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
d = {'b':0, 't':1, 'e':2, 'm':3}
y_train = train.iloc[:,0].replace(d)
y_train.to_csv('data/chap8/y_train.txt',header=False, index=False)
y_valid = valid.iloc[:,0].replace(d)
y_valid.to_csv('data/chap8/y_valid.txt',header=False, index=False)
y_test = test.iloc[:,0].replace(d)
y_test.to_csv('data/chap8/y_test.txt',header=False, index=False)

In [ ]:
def write_X(file_name, df):
    with open(file_name, 'w') as f:
        for text in df.iloc[:, 1]:
            vectors = []
            for word in text.split():
                if word in model.vocab:
                    vectors.append(model[word])
            if (len(vectors)==0):
                vector = np.zeros(300)
            else:
                vectors = np.array(vectors)
                vector = vectors.mean(axis=0)
            vector = vector.astype(np.str).tolist()
            output = ' '.join(vector)+'\n'
            f.write(output)

In [ ]:
write_X('data/chap8/X_train.txt', train)
write_X('data/chap8/X_valid.txt', valid)
write_X('data/chap8/X_test.txt', test)

### 71. 単層ニューラルネットワークによる予測

In [ ]:
X_train = np.loadtxt('data/chap8/X_train.txt', delimiter=' ')
X_train = torch.tensor(X_train, dtype=torch.float32)
W = torch.randn(300, 4, requires_grad=True)
softmax = torch.nn.Softmax(dim=1)
print (softmax(torch.matmul(X_train[:1], W)))
print (softmax(torch.matmul(X_train[:4], W)))

### 72. 損失と勾配の計算

In [ ]:
y_train = np.loadtxt('data/chap8/y_train.txt')
y_train = torch.tensor(y_train, dtype=torch.int64)
loss = torch.nn.CrossEntropyLoss()
print (loss(torch.matmul(X_train[:1], W),y_train[:1]))
print (loss(torch.matmul(X_train[:4], W),y_train[:4]))

In [ ]:
ans = [] # 以下、確認
for s,i in zip(softmax(torch.matmul(X_train[:4], W)),y_train[:4]):
    ans.append(-np.log(s[i].detach()))
print (np.mean(ans))

### 73. 確率的勾配降下法による学習

In [ ]:
from tqdm import tqdm

In [ ]:
W_73 = W
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD([W_73], lr=0.01)

for epoch in range(100):
    optimizer.zero_grad()
    y = torch.matmul(X_train, W_73)
    loss = loss_fn(y, y_train)
    loss.backward()
    optimizer.step()

In [ ]:
X_train.shape
W.shape

In [ ]:
ans = [] # 以下、確認
for s,i in zip(softmax(torch.matmul(X_train[:4], W_73)),y_train[:4]):
    ans.append(-np.log(s[i].detach()))
print (np.mean(ans))

### 74. 正解率の計測

In [ ]:
def accuracy(pred, label):
    pred = np.argmax(pred.data.numpy(), axis=1)
    label = label.data.numpy()
    return (pred == label).mean()

In [ ]:
X_valid = np.loadtxt('data/chap8/X_valid.txt', delimiter=' ')
X_valid = torch.tensor(X_valid, dtype=torch.float32)
y_valid = np.loadtxt('data/chap8/y_valid.txt')
y_valid = torch.tensor(y_valid, dtype=torch.int64)

pred = torch.matmul(X_valid, W_73)
print(accuracy(pred, y_valid))

### 75. 損失と正解率のプロット

In [ ]:
%load_ext tensorboard
!rm -rf ./runs
%tensorboard --logdir ./runs
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [ ]:
W_75 = W
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD([W_75], lr=0.01)

for epoch in range(1000):
    optimizer.zero_grad()
    y = torch.matmul(X_train, W_75)
    loss = loss_fn(y, y_train)
    loss.backward()
    optimizer.step()
    with torch.no_grad():
        y_pred = torch.matmul(X_train, W_75)
        loss = loss_fn(y_pred, y_train) 
        writer.add_scalar('Loss/train', loss, epoch)
        writer.add_scalar('Accuracy/train', accuracy(y_pred, y_train), epoch)
        y_pred = torch.matmul(X_valid, W_75)
        loss = loss_fn(y_pred, y_valid)
        writer.add_scalar('Loss/valid', loss, epoch)
        writer.add_scalar('Accuracy/valid', accuracy(y_pred,y_valid), epoch)

    

### 76. チェックポイント

In [ ]:
W_76 = W
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD([W_76], lr=0.01)

for epoch in range(1000):
    optimizer.zero_grad()
    y = torch.matmul(X_train, W_76)
    loss = loss_fn(y, y_train)
    loss.backward()
    optimizer.step()
    with torch.no_grad():
        y_pred = torch.matmul(X_train, W_76)
        loss = loss_fn(y_pred, y_train) 
        y_pred = torch.matmul(X_valid, W_76)
        loss = loss_fn(y_pred, y_valid)
        if epoch%100 == 0:
            torch.save(W, f'data/chap8/{str(epoch)}.model')
            torch.save(optimizer.state_dict(), f'data/chap8/{str(epoch)}.parm')

### 77. ミニバッチ化

In [ ]:
import time
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
class DataSet:
    def __init__(self, X, y):
        self.X = X # 入力
        self.t = y # 出力

    def __len__(self):
        return len(self.X) # データ数(10)を返す

    def __getitem__(self, index):
        # index番目の入出力ペアを返す
        return self.X[index], self.t[index]

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
W_77 = W

dataset = DataSet(X_train, y_train)
ls_bs = [2**i for i in range(15)]
ls_time = []
for bs in ls_bs:
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=bs, shuffle=True)
    optimizer = torch.optim.SGD([W_77], lr=0.01)
    for epoch in range(1):
        start = time.time()
        for data in dataloader:
            optimizer.zero_grad()
            X = data[0]
            y = torch.matmul(X, W_77)
            loss = loss_fn(y, data[1])
            loss.backward()
            optimizer.step()
        ls_time.append(time.time()-start)
print(ls_time)

### 79. 多層ニューラルネットワーク

In [ ]:
!git add .

In [ ]:
!git commit -m "update chapter8"

In [ ]:
!git push